# Time Series Models Experiments

#### (1) LSTM Model

In [5]:
from tensorflow.keras.optimizers import Adam
import streamlit as st
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import datetime
from datetime import date, timedelta
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from prophet import Prophet
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler



In [2]:
df= pd.read_csv('AAPL_stocks.csv')
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-01-04,7.622500,7.660714,7.585000,7.643214,6.470740,493729600
1,2010-01-05,7.664286,7.699643,7.616071,7.656429,6.481928,601904800
2,2010-01-06,7.656429,7.686786,7.526786,7.534643,6.378823,552160000
3,2010-01-07,7.562500,7.571429,7.466071,7.520714,6.367033,477131200
4,2010-01-08,7.510714,7.571429,7.466429,7.570714,6.409362,447610800
...,...,...,...,...,...,...,...
3576,2024-03-20,175.720001,178.669998,175.089996,178.669998,178.669998,53423100
3577,2024-03-21,177.050003,177.490005,170.839996,171.369995,171.369995,106181300
3578,2024-03-22,171.759995,173.050003,170.059998,172.279999,172.279999,71106600
3579,2024-03-25,170.570007,171.940002,169.449997,170.850006,170.850006,54235800


In [3]:
df.dtypes

Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object

### 1st method (LSTM)

In [21]:

## Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

## Sort DataFrame by date
df = df.sort_values('Date')

## Extract 'Open' prices
data = df['Open'].values.reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

## Split data into train and test sets (80% train, 20% test)
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]

## Function to create dataset with lookback
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        X.append(dataset[i:(i + look_back), 0])
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

## Create train and test datasets with lookback
look_back = 60  # Adjust this value for your needs
X_train, y_train = create_dataset(train_data, look_back)
X_test, y_test = create_dataset(test_data, look_back)

## Reshape input data to [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

## Build LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

## Train the model
model.fit(X_train, y_train, batch_size=64, epochs=100)

# # Make predictions
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)

# Plot the results
fig = go.Figure()

# Plot training data
fig.add_trace(go.Scatter(x=df['Date'][:train_size], y=data[:train_size].flatten(),
                    mode='lines', name='Training Data', line=dict(color='blue')))

# Plot testing data
fig.add_trace(go.Scatter(x=df['Date'][train_size+look_back:], y=data[train_size+look_back:].flatten(),
                    mode='lines', name='Testing Data', line=dict(color='green')))

# Plot predicted data
fig.add_trace(go.Scatter(x=df['Date'][train_size+look_back:], y=predictions.flatten(),
                    mode='lines', name='Predicted Data', line=dict(color='red')))

fig.update_layout(title='Stock Price Prediction using LSTM',
                   xaxis_title='Date',
                   yaxis_title='Stock Price')

fig.show()

# # Save the model
# model.save('stock_price_prediction_model.h5')


Epoch 1/100


c:\Users\zeesh\anaconda3\envs\forecasting_env\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



44/44 ━━━━━━━━━━━━━━━━━━━━ 6s 72ms/step - loss: 0.0114
Epoch 2/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 64ms/step - loss: 2.3835e-04
Epoch 3/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 60ms/step - loss: 1.6563e-04
Epoch 4/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 5s 58ms/step - loss: 1.2977e-04
Epoch 5/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - loss: 1.4301e-04
Epoch 6/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - loss: 1.6745e-04
Epoch 7/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - loss: 1.4311e-04
Epoch 8/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - loss: 1.4968e-04
Epoch 9/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 60ms/step - loss: 1.5887e-04
Epoch 10/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 60ms/step - loss: 1.3373e-04
Epoch 11/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step - loss: 1.3318e-04
Epoch 12/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - loss: 1.2006e-04
Epoch 13/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - loss: 1.1657e-04
Epoch 14/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - loss: 1.0364e-04
Epoch 15/100
44/4

### 2nd Method (LSTM)

In [20]:
# Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Sort DataFrame by date
df = df.sort_values('Date')

# Extract 'Open' prices
data = df['Open'].values.reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Split data into train and test sets (80% train, 20% test)
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]

# Function to create dataset with lookback
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        X.append(dataset[i:(i + look_back), 0])
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

# Create train and test datasets with lookback
look_back = 60  # Adjust this value for your needs
X_train, y_train = create_dataset(train_data, look_back)
X_test, y_test = create_dataset(test_data, look_back)

# Reshape input data to [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Build LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, batch_size=64, epochs=100)

# Define the number of days to predict
days_to_predict = 90  # Adjust this value for your needs

# Make predictions for the test dataset
test_predictions = model.predict(X_test)
test_predictions = scaler.inverse_transform(test_predictions)

# Make predictions for the specified additional number of days
predictions = []
current_batch = X_test[-1]

for i in range(days_to_predict):
    current_pred = model.predict(current_batch.reshape(1, look_back, 1))
    predictions.append(current_pred[0][0])  # Extract the scalar value from the prediction
    current_batch = np.append(current_batch[1:], current_pred[0][0])  # Append scalar prediction to current batch

extended_predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

# Plot the results
fig = go.Figure()

# Plot training data
fig.add_trace(go.Scatter(x=df['Date'][:train_size], y=data[:train_size].flatten(),
                    mode='lines', name='Training Data', line=dict(color='blue')))

# Plot testing data
fig.add_trace(go.Scatter(x=df['Date'][train_size+look_back:], y=data[train_size+look_back:].flatten(),
                    mode='lines', name='Testing Data', line=dict(color='green')))

# Plot predicted data for test dataset
fig.add_trace(go.Scatter(x=df['Date'][train_size+look_back:train_size+look_back+len(test_predictions)],
                    y=test_predictions.flatten(), mode='lines',
                    name='Predicted Data (Test Dataset)', line=dict(color='orange')))

# Plot extended predicted data
fig.add_trace(go.Scatter(x=pd.date_range(start=df['Date'].iloc[-1], periods=days_to_predict+1)[1:],
                    y=extended_predictions.flatten(),
                    mode='lines', name=f'Extended Predicted Data ({days_to_predict} days)', line=dict(color='red')))

fig.update_layout(title='Stock Price Prediction using LSTM',
                   xaxis_title='Date',
                   yaxis_title='Stock Price')

fig.show()




Epoch 1/100


c:\Users\zeesh\anaconda3\envs\forecasting_env\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



44/44 ━━━━━━━━━━━━━━━━━━━━ 6s 60ms/step - loss: 0.0113
Epoch 2/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - loss: 2.7018e-04
Epoch 3/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - loss: 1.5432e-04
Epoch 4/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - loss: 1.5731e-04
Epoch 5/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - loss: 1.2994e-04
Epoch 6/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - loss: 1.3963e-04
Epoch 7/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - loss: 1.4118e-04
Epoch 8/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - loss: 1.3188e-04
Epoch 9/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - loss: 1.3276e-04
Epoch 10/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - loss: 1.2532e-04
Epoch 11/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 57ms/step - loss: 1.2537e-04
Epoch 12/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 64ms/step - loss: 2.0094e-04
Epoch 13/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 65ms/step - loss: 1.2836e-04
Epoch 14/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - loss: 1.0566e-04
Epoch 15/100
44/4